# PV System Heatmap

In [1]:
# pyathena
from pyathena.connection import Connection
from pyathena.pandas_cursor import PandasCursor

In [2]:
cursor = Connection(
    profile_name="dev",
    region_name="us-west-2",
    s3_staging_dir="s3://nrel-tests/tracking_the_sun/"
).cursor(PandasCursor)

## State Level

In [3]:
# bokeh
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, PrintfTickFormatter, ColorBar, LogColorMapper, LogTicker
from bokeh.plotting import figure, show

In [4]:
pv_state_year = cursor.execute(
    """
    SELECT state, CAST(YEAR(date_parse(installation_date, '%c/%e/%y')) AS VARCHAR(4)) AS year, COUNT(*) as count
    FROM oedi.oedi_dev_tracking_the_sun
    GROUP BY CAST(YEAR(date_parse(installation_date, '%c/%e/%y')) AS VARCHAR(4)), state;
    """
).as_pandas()

In [5]:
output_notebook()

years = sorted(pv_state_year["year"].unique())
states = list(reversed(sorted(pv_state_year["state"].unique())))

ifig = figure(
    title="Heatmap of State PV Systems",
    x_range=years,
    y_range=states,
    sizing_mode="stretch_both",
    x_axis_location="above",
    toolbar_location='below',
    tooltips=[("state", "@state"), ("year", "@year"), ("count", "@count")]
)

colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LogColorMapper(
    palette=colors,
    low=pv_state_year["count"].min(),
    high=pv_state_year["count"].max()
)
ifig.rect(
    x="year",
    y="state",
    width=1,
    height=1,
    source=pv_state_year,
    fill_color={"field": "count", "transform": mapper},
    line_color=None
)
ifig.grid.grid_line_color = None
ifig.axis.axis_line_color = None
ifig.axis.major_tick_line_color = None
ifig.axis.major_label_text_font_size = "6pt"
ifig.axis.major_label_standoff = 0

color_bar = ColorBar(
    color_mapper=mapper,
    major_label_text_font_size="6pt",
    ticker=LogTicker(),
    formatter=PrintfTickFormatter(format="%d"),
    label_standoff=6,
    border_line_color=None,
    location=(0, 0),
)

ifig.add_layout(color_bar, 'right')


show(ifig)

Loading BokehJS ...